In [14]:
import logging
import tensorflow as tf
import keras

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

tf.__version__

'2.4.1'

In [73]:
import tensorflow_datasets as tfds
from tensorflow.train import Feature, Example, Features, BytesList, Int64List
from pathlib import Path

def ch13_ex_9():
    train_fraction = 0.99
    seed = 101
    train_dir = 'data/train'
    val_dir = 'data/val'

    
    def create_dataset(dataset, data_dir):
        filepaths = []
        Path(data_dir).mkdir(parents=True, exist_ok=True)
        for i, d in enumerate(dataset):
            ser_image = tf.io.serialize_tensor(d['image']).numpy()
            image_example = Example(
                features=Features(
                    feature={
                        'image': Feature(bytes_list=BytesList(value=[ser_image])),
                        'label': Feature(int64_list=Int64List(value=[d['label']]))
                    }
                )
            )
            filepath = f'{data_dir}/image{i}.tfrecord'
            with tf.io.TFRecordWriter(filepath) as f:
                f.write(image_example.SerializeToString())
            filepaths.append(filepath)
        return filepaths
                
    def create_pipeline(filepaths):
        image_descr = {
            'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
            'label': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
        }
        def parse_image_example(path):
            parsed_image_example = tf.io.parse_single_example(ser, image_descr)
            image = tf.io.parse_tensor(parsed_image_example["image"], out_type=tf.uint8)
#             image = tf.reshape(image, shape=[28, 28])
            image = tf.data.Dataset.from_tensor_slices([image])
            return image
       
        for ser in tf.data.TFRecordDataset(filepaths):
            break
#         dataset = tf.data.Dataset.list_files(filepaths, seed=seed).interleave(lambda path: tf.io.parse_single_example(path, image_descr), 
#                                                                               num_parallel_calls=tf.data.AUTOTUNE)
        dataset = tf.data.TFRecordDataset(filepaths).interleave(parse_image_example, num_parallel_calls=tf.data.AUTOTUNE).take(1)
        for d in dataset:
            log.info(d)
        log.info(f'Dataset: {dataset}')
    
    dataset = tfds.load("fashion_mnist")
    log.info(f'Loaded dataset: {dataset}')
    train_idx = int(len(dataset['train']) * train_fraction)
    train = dataset['train'].take(train_idx).shuffle(101, seed=seed, reshuffle_each_iteration=True)
    log.info(f'Train dataset len: {len(train)}')
    val = dataset['train'].skip(train_idx)
    log.info(f'Val dataset len: {len(val)}')
#     create_dataset(train, data_dir=train_dir)
    val_filepaths = create_dataset(dataset=val, data_dir=val_dir)
    create_pipeline(val_filepaths)

             
ch13_ex_9()

2021-04-18 10:40:14,271 : INFO : Load dataset info from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-18 10:40:14,273 : INFO : Reusing dataset fashion_mnist (/Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1)
2021-04-18 10:40:14,274 : INFO : Constructing tf.data.Dataset for split None, from /Users/mkhokhlush/tensorflow_datasets/fashion_mnist/3.0.1
2021-04-18 10:40:14,321 : INFO : Loaded dataset: {'train': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>, 'test': <PrefetchDataset shapes: {image: (28, 28, 1), label: ()}, types: {image: tf.uint8, label: tf.int64}>}
2021-04-18 10:40:14,323 : INFO : Train dataset len: 59400
2021-04-18 10:40:14,323 : INFO : Val dataset len: 600
2021-04-18 10:40:15,688 : INFO : tf.Tensor(
[[[  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [  0]
  [168]
  [220]
  [197]
  [202]
  [203]
  [193]
  [196]
  [197]
  [190]
  [230]
  [104]
  [  0]
  [  0]
  [  0]
  [  0]
  [